In [54]:
#encoding=utf-8
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

# get tfidf

In [55]:
# path = '/home/kai/data/jiangning/Chatbot_1/Chatbot1.0/data/'
path = '../../data/IDClassifier/'
data = pd.read_csv(path + 'cleaned_mock_up_data.csv', encoding='utf8')
print(data.shape)
data.head(10)

(1435, 3)


,label,split_text,text
0,0,喜喜,喂唉你好
1,0,我 知道 你 来 要钱 我 没 钱,知道你是来催钱的，老子没钱
2,0,什么,啊
3,0,你 收集 公司 的 工作人员 吗 ？,你是催收公司的工作人员么
4,1,我 是 李 大宝 不是 张 的 狗 。,我是李大宝，不是张二狗
5,2,谁,谁
6,1,我 不是 你 想要 的,我不是你要找的人
7,0,好,嗯
8,2,你 重复 了 它 你 没有 听到 它 。,你重新说一下，刚才没听清
9,0,唉呀,唉呀


In [56]:
# phrase_vectorizer1 = TfidfVectorizer(ngram_range=(1,3),
#                                     strip_accents='unicode', 
#                                     max_features=100000, 
#                                     analyzer='word',
#                                     sublinear_tf=True,
#                                     token_pattern=r'\w{1,}')
# data.split_text.iloc[:2]
# dd = ['I like apple do you like it', 'I do not like banana']
# phrase_vectorizer1.fit(dd)

# print('transform phrase')
# phrase = phrase_vectorizer1.transform(dd)


In [57]:
data.split_text.iloc[1]

'我 知道 你 来 要钱   我 没 钱'

In [58]:
# phrase_vectorizer1.vocabulary_

In [59]:
phrase_vectorizer = TfidfVectorizer(ngram_range=(1,3),
                                    strip_accents='unicode', 
                                    max_features=100000, 
                                    analyzer='word',
                                    sublinear_tf=True,
                                    token_pattern=r'\w{1,}')

print('fitting phrase')
phrase_vectorizer.fit(data.split_text)

print('transform phrase')
phrase = phrase_vectorizer.transform(data.split_text)

phrase

fitting phrase
transform phrase


<1435x3789 sparse matrix of type '<class 'numpy.float64'>'
	with 14364 stored elements in Compressed Sparse Row format>

# SVC

In [60]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC

l_svc = LinearSVC()
clf = CalibratedClassifierCV(l_svc) 
clf.fit(phrase, data.label)
print(clf.score(phrase, data.label))

0.970034843206


In [61]:
print(clf.predict(phrase))

[0 0 0 ..., 0 0 0]


# logistic

In [62]:
from sklearn.linear_model import LogisticRegression

In [63]:
log_r = LogisticRegression()
log_r.fit(phrase, data.label)
print(log_r.score(phrase, data.label))

0.926132404181


In [64]:
print(log_r.predict(phrase))

[0 0 0 ..., 0 0 0]


In [65]:
# l_svc.predict(phrase)-log_r.predict(phrase)

# lightGBM

In [66]:
import lightgbm as lgb

In [67]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(data.label.values)
onelabels = le.transform(data.label.values)

In [68]:
onelabels

array([0, 0, 0, ..., 0, 0, 0])

In [69]:
from sklearn.preprocessing import MultiLabelBinarizer
multicoder = MultiLabelBinarizer()
lables = multicoder.fit_transform([data.label.values])

In [70]:
lables

array([[1, 1, 1]])

In [71]:
data.label.values

array([0, 0, 0, ..., 0, 0, 0])

In [72]:
# lables.todense()

In [73]:
params = {
    'learning_rate': 0.2,
    'num_iterations':1000,
    'application': 'multiclassova',
    'num_class': 3,
    'num_leaves': 31,
    'verbosity': -1,
    'metric': 'multi_error',
    'data_random_seed': 2,
#     'bagging_fraction': 0.8,
#     'feature_fraction': 0.6,
    'nthread': 4,
    'lambda_l1': 1,
    'lambda_l2': 1,
    'early_stopping_round':200
} 

# lgbm_train = lgb.Dataset(phrase, data.label)
lgbm_train = lgb.Dataset(phrase, onelabels)
lgbm_val = lgb.Dataset(phrase, onelabels)
lgbm_model = lgb.train(params,lgbm_train,valid_sets=lgbm_val, verbose_eval=5)

Training until validation scores don't improve for 200 rounds.
[5]	valid_0's multi_error: 0.148432
[10]	valid_0's multi_error: 0.12892
[15]	valid_0's multi_error: 0.110801
[20]	valid_0's multi_error: 0.100348
[25]	valid_0's multi_error: 0.0885017
[30]	valid_0's multi_error: 0.0815331
[35]	valid_0's multi_error: 0.0808362
[40]	valid_0's multi_error: 0.0745645
[45]	valid_0's multi_error: 0.0738676
[50]	valid_0's multi_error: 0.0731707
[55]	valid_0's multi_error: 0.0724739
[60]	valid_0's multi_error: 0.0710801
[65]	valid_0's multi_error: 0.0675958
[70]	valid_0's multi_error: 0.0675958
[75]	valid_0's multi_error: 0.066899
[80]	valid_0's multi_error: 0.0655052
[85]	valid_0's multi_error: 0.0648084
[90]	valid_0's multi_error: 0.0634146
[95]	valid_0's multi_error: 0.0634146
[100]	valid_0's multi_error: 0.0634146
[105]	valid_0's multi_error: 0.0634146
[110]	valid_0's multi_error: 0.0620209
[115]	valid_0's multi_error: 0.061324
[120]	valid_0's multi_error: 0.0627178
[125]	valid_0's multi_error:

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [74]:
print(lgbm_model.predict(phrase))

[[ 0.73547642  0.04426599  0.18111404]
 [ 0.98311386  0.01027268  0.00852655]
 [ 0.61446525  0.03224424  0.36991184]
 ..., 
 [ 0.73547642  0.04426599  0.18111404]
 [ 0.99648274  0.00323028  0.00292047]
 [ 0.72330022  0.18173076  0.0093197 ]]


In [75]:
# lgbm_model.predict(phrase)

In [76]:
phrase

<1435x3789 sparse matrix of type '<class 'numpy.float64'>'
	with 14364 stored elements in Compressed Sparse Row format>

# Save

In [77]:
import pickle

# save tfidf
pickle.dump(phrase_vectorizer, open("../../savedModel/IDClassifier/tfidf.pickle", "wb"))
# pickle.dump(train_comment_features, open("train_comment_features.pickle", "wb"))
# pickle.dump(test_comment_features, open("test_comment_features.pickle", "wb"))

# save linear svc
pickle.dump(clf, open("../../savedModel/IDClassifier/LinearSVC.pickle", "wb"))
# save logistic
pickle.dump(log_r, open("../../savedModel/IDClassifier/Logistic.pickle", "wb"))
# save lightGBM
pickle.dump(lgbm_model, open("../../savedModel/IDClassifier/Lgbm.pickle", "wb"))

In [78]:
# pickle.load(open("../../savedModel/IDClassifier/Lgbm.pickle", 'rb'))

# test

In [79]:
import jieba
sentence = '对啊'
sentence = jieba.cut(sentence, cut_all = False)
sentence = ' '.join(sentence)
test = phrase_vectorizer.transform([sentence])
# test = phrase_vectorizer.transform(['我 在 洗澡'])


In [80]:
test

<1x3789 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [81]:
clf.predict_proba(test) # linear svc

array([[ 0.97165868,  0.00946117,  0.01888016]])

In [82]:
log_r.predict_proba(test) # logistic

array([[ 0.87116412,  0.07289443,  0.05594145]])

In [83]:
lgbm_model.predict(test) # light gbm

array([[ 0.99303539,  0.00546324,  0.00490804]])

# labeling

In [84]:
# basic logic: find the max probability of 3 models, if it is larger than threshold, return the corresponding label, otherwise, return 2 (others).
result = np.vstack((clf.predict_proba(test),log_r.predict_proba(test),lgbm_model.predict(test)))
pos = np.where(result == np.max(result))

threshold = 0.7
if np.max(result)<threshold:
    label = 2
else:
    label = pos[1]
    label = label[0]
    
print('label=',label)
print('prob=',np.max(result))

label= 0
prob= 0.993035390427


# Save the whole thing

In [100]:
# import jieba
# import numpy as np

# class IDClassifier1:
    
#     def __init__(self, **model):
#         """
#         suggested parameters:
#         svc, logistic, lightgbm, jieba_path,tfidf
#         """
#         self._load_model(**model)
        
#     def _load_model(self,**model):
#         self.svc = model.get('svc')
#         self.logistic = model.get('logistic')
#         self.lightgbm = model.get('lightgbm')
#         self.tfidf = model.get('tfidf')
#         # load jieba
#         jieba_path = model.get('jieba_path')
#         if jieba_path is not None:
#             jieba.load_userdict(jieba_path)
        
        
#     def classify(self, sentence):
#         sentence = jieba.cut(sentence, cut_all = False)
#         sentence = ' '.join(sentence)
#         matrix = self.tfidf.transform([sentence])
#         result = np.vstack((self.svc.predict_proba(matrix),
#                             self.logistic.predict_proba(matrix),
#                             self.lightgbm.predict(matrix)))
#         max_pred = np.max(result, axis=0)
#         max_arg = np.argmax(max_pred)
#         threshold = 0.6
#         if np.max(max_pred)<threshold:
#             label = 2
#             print(max_pred)
#         else:
#             label = max_arg
#         return (label, np.max(max_pred))
     
        

In [101]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [102]:
from IDClassifier_py import IDClassifier
idc = IDClassifier(svc=clf, logistic=log_r, lightgbm=lgbm_model, tfidf=phrase_vectorizer, jieba_path='../WordCut/userdict.txt')

pickle.dump(idc, open("../../savedModel/IDClassifier/IDClassifier.pickle", "wb"))

In [103]:
## when apply, need to import sys path of the .py file, then load pkl file 